# Estimación de peso y dimensiones de los envíos de Mercado Libre

# Materia: Análisis y curación de datos

## Análisis del dataset

Se pide: A partir de lo visto en la teoría de la materia y del segundo laboratorio, diagramar una comunicación en formato textual o interactivo describiendo la solución de las actividades propuestas a continuación. Al final de las mismas se proveen actividades opcionales (no obligatorias) que pueden resultar de interés.

### Actividades Propuestas:
    1.	Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.

    2.	Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.

    3.	Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

    4.	Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.

    5.	Transformar variables categóricas en números (Se recomienda OneHotEncoding) para las columnas (Sugerencia: arrancar con un sample de ~10K items)

        a.	`CATALOG_PRODUCT_ID`
        b.	`CONDITION`
        c.	`DOMAIN_ID`
        d.	`SELLER_ID`
        e.	`BRAND` (extraída en 4)
        f.	`MODEL`(extraída en 4)

    6.	En caso de tener alguna variable no medida (en nuestro caso `PRICE`) imputar sus valores utilizando kNN.

    7.	Medir las distribuciones de las variables como histogramas, realizar normalizaciones e identificar outliers con los métodos vistos en clase. Hacer análisis de estos outliers y considerar si sería correcto o no eliminarlos del dataset. Sugerencia: Identificar outliers de las columnas `SHP_WEIGHT` y `SHP_VOLUME`, donde `SHP_VOLUME` se define como el producto de las dimensiones.

    8.	[Opcional] Si están interesados en usar texto y técnicas de NLP sobre el título:
        a.	Realizar preprocesamiento del texto, se recomienda leer este artículo.
        b.	Para convertir el título en vectores se recomienda ver tf-idf. wiki, sklearn.

    9.	[Opcional] Aplicar PCA y decidir si existe un subespacio del espacio de features donde viven los datos y son bien descriptos. Graficar.

    10.	[Opcional] Aplicar Mixturas Gaussianas (GMM) para hacer clustering. 

La comunicación debe estar apuntada a un público técnico pero sin conocimiento del tema particular, como por ejemplo, sus compañeros de clase o stakeholders del proyecto. Idealmente, además del documento se debería generar una presentación corta para stakeholders explicando el análisis realizado sobre los datos y las conclusiones obtenidas de tal análisis.

Se evaluarán los siguientes aspectos:

    ●	El informe debe contener un mensaje claro y presentado de forma concisa.
    ●	Los gráficos deben aplicar los conceptos de percepción visual vistos en clase.
    ●	Se debe describir o estimar la significancia estadística de su trabajo.


In [39]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy
import seaborn
import scipy as sc

In [40]:
random.seed(0)

El dataset corresponde a un muestreo de envíos durante tres meses de 2018 en MLB (Mercado Libre Brasil), filtrados según métodos de envíos y por carriers (empresas de correos a cargo del envío propiamente dicho). A su vez, fue muestreado de forma no uniforme para romper de forma manual la distribución de los datos.
Los datos son:[meli_dataset_20190426.csv] (https://drive.google.com/open?id=1tNUKD1lf1z8C7LPpCruiDl6SKBAHb79v).

In [41]:
DATASET = 'meli_dataset_20190426.csv'

In [42]:
df_original = pd.read_csv(DATASET, low_memory=False)


In [43]:
df= df_original

In [ ]:
df = df.head(10000) 

In [44]:
df.columns

Index(['ITEM_ID', 'SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT',
       'ATTRIBUTES', 'CATALOG_PRODUCT_ID', 'CONDITION', 'DOMAIN_ID', 'PRICE',
       'SELLER_ID', 'STATUS', 'TITLE'],
      dtype='object')

#### ¿Qué significa cada columna?

* ITEM_ID: id unívoco de cada item publicado. (Ofuscado)
* SHP_WEIGHT: peso del paquete informado por el correo.
* SHP_LENGTH: largo del paquete informado por el correo.
* SHP_WIDTH: ancho del paquete informado por el correo.
* SHP_HEIGHT: altura del paquete informado por el correo.
* ATTRIBUTES: atributos como marca y modelo, entre otros, en formato json-lines
* CATALOG_PRODUCT_ID: id del catálogo (ofuscado).
* CONDITION: condición de venta (nuevo o usado).
* DOMAIN_ID: id de la categoría a la que pertenece la publicación.
* PRICE: precio en reales.
* SELLER_ID: id del vendedor (ofuscado).
* STATUS: estado de la publicación (activa, cerrada, pausada, etc.)
* TITLE: título de la publicación.

In [45]:
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,STATUS,TITLE
490242,Z42BBD9ZJV,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",S48H0J215Z,new,MLB-CAMERA_TRIPODS,63.00,LZCD9O40J5,active,"Tripé Celular Camera 1,30 M + Kit Self + Mini ..."
133189,MUZ6M23HYV,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-DIAPER_BAGS,85.99,JIZHPHTAUF,active,Kit Bolsa Maternidade Bebê Azm Trocador Menino...
129593,DZE3NVYEUU,2210.0,67.0,43.0,8.0,"[{'id': 'CAR_BRAND', 'name': 'Marca do carro',...",H53U1H7Q5G,new,MLB-DIECAST_CARS,45.00,WCQCUQ0ZZ0,active,Estante Em Mdf Ultra Para 48 Carrinhos Promoç...
63941,YB9NEGL73I,20.0,25.0,11.0,5.0,"[{'id': 'ACCESSORY_TYPE', 'name': 'Tipo de ace...",H53U1H7Q5G,new,MLB-CELLPHONE_SOFTWARE,8.99,ZWFWFMIWO5,paused,"R-sim 9 Pro Gevey Desbloqueador Iphone 4s, 5 E 5c"
229172,OM9G4YU0SO,6020.0,20.0,15.0,10.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-LIGHT_BULBS,845.49,Q7J9ABS88D,paused,Kit 100 Lâmpada Bulbo 12w Super Led Bivolt Bra...


## Actividad 1:

Eliminar valores cuyo status sea `404` , luego eliminar la columna `status` del dataset ya que solo es útil para limpieza.


In [46]:
# df.filter(["STATUS"])!= "404"

df = df[df.STATUS != "404"]
df = df.drop(columns=['STATUS'])
df.sample(5)

,ITEM_ID,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,ATTRIBUTES,CATALOG_PRODUCT_ID,CONDITION,DOMAIN_ID,PRICE,SELLER_ID,TITLE
24947,AG7EL1FYDG,149.0,25.0,22.0,5.0,"[{'id': 'BRAND', 'name': 'Marca da película', ...",H53U1H7Q5G,new,MLB-CELLPHONE_AND_TABLET_SCREEN_PROTECTORS,25.60,CHLAXR8KFM,Pelicula De Vidro Proteção Camara+ Frontal Par...
153097,GNTE5OTA2Z,500.0,25.0,11.0,5.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-AUTOMOTIVE_CELLPHONE_MOUNTS,24.99,XHY7BSZ6O6,Suporte Fixo Articulado 70cm Celular Gps Berço...
138377,ITQWE40IBS,84.0,25.0,16.0,13.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-HOME_APPLIANCES,35.65,VW3PIFFAGM,Filtro Bem Estar Consul Original W10515645
306493,P54YW37EZD,NaN,NaN,NaN,NaN,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-MANICURE_AND_PEDICURE_SUPPLIES,299.90,M5QJ1Y7ZQC,Kit Manicure Promoção Cirandinha + Cadeira Cl...
237372,C26B7GLVE3,19.0,0.0,0.0,0.0,"[{'id': 'BRAND', 'name': 'Marca', 'value_id': ...",H53U1H7Q5G,new,MLB-HEALTH_CARE_SUPPLIES,15.99,KMYJRXAZIR,* Ofertão * Dilatador Nasal Respirar Fácil Ant...


## Actividad 2:

Eliminar los valores NaN de las columnas con prefijo `SHP_`. Estas son aquellas que representan o peso o dimensiones de un item.


In [47]:
# df = df.dropna(subset=['SHP_WEIGHT'])
# df = df.dropna(subset=['SHP_LENGTH'])
# df = df.dropna(subset=['SHP_WIDTH'])
# df = df.dropna(subset=['SHP_HEIGHT'])

# o bien:
df = df.dropna(subset=['SHP_WEIGHT', 'SHP_LENGTH', 'SHP_WIDTH', 'SHP_HEIGHT'])

#o bien

# df= df.dropna(df.filter(regex ='SHP_'))


#df.sample(3)



## Actividad 3:

Agrupar por item id y calcular mediana de peso y medidas. De esta forma debería quedar una única fila por cada item_id.

In [51]:
medianas = df.groupby('ITEM_ID').median()
medianas


,SHP_WEIGHT,SHP_LENGTH,SHP_WIDTH,SHP_HEIGHT,PRICE
ITEM_ID,,,,,
A002DG7EAZ,812.0,36.0,32.0,12.0,NaN
A00SG33UIH,2320.0,16.0,11.0,4.0,149.99
A00VIC9XL7,213.0,16.0,13.0,10.0,329.00
A00VM7MP9F,175.0,25.0,20.0,15.0,45.00
A00W1VSE3K,82.0,30.0,15.0,5.0,31.98
...,...,...,...,...,...
ZZZ3SAO9N3,2030.0,60.0,40.0,20.0,349.00
ZZZMIXSNBZ,8030.0,52.0,52.0,23.0,759.99
ZZZOE5LENH,876.0,25.0,25.0,5.0,129.90


## Actividad 4:

Parsear la columna de atributos y extraer a columnas propias aquellos atributos cuyo `id` sea `BRAND` o `MODEL`. Estos atributos representan marca o modelo que el vendedor del item ingresó en la publicación. [Opcional] No es necesario limitarse a estos dos atributos, se puede probar quedarse con los N atributos más frecuentes.